In [ ]:
import os
import cv2
from datetime import date
import json
import itertools

import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel
import utm
import pandas as pd

In [ ]:
geojson_path = "./data/target_zone.geojson"

with open("./secrets/sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"], 
    credentials["password"],
    "https://scihub.copernicus.eu/dhus"
)

shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2020, 7, 27), date(2020, 7, 29)),
    platformname="Sentinel-2",
    processinglevel="Level-2A",
    cloudcoverpercentage=(0, 30)
)

images_df = api.to_dataframe(images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
images_df[key_columns].head()

In [ ]:
uuid = "4172581f-67cf-4c08-9250-be29c86bfae7"
# uuid = "a6889608-c2b4-4b0d-a731-a7f0c2da8951"
# api.download(uuid)

In [ ]:
# article file
image_folder = "./data/S2B_MSIL2A_20200727T105619_N0214_R094_T30TXQ_20200727T144035.SAFE"
# project file
# image_folder = "./data/S2B_MSIL2A_20200728T070629_N0214_R106_T38MME_20200728T113142.SAFE"

In [ ]:
def get_band(image_folder, band, resolution=10):
    subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    image_folder_path  = f"{image_folder}/GRANULE/{subfolder}/IMG_DATA/R{resolution}m"
    image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    selected_file = [im for im in image_files if im.split("_")[2] == band][0]
    
    with rasterio.open(f"{image_folder_path}/{selected_file}") as infile:
        img = infile.read(1)
    
    return img

In [ ]:
def get_band_bis(img_folder, band):
    image_files = [im for im in os.listdir(img_folder) if im[-4:] == ".jp2"]
    selected_file = [im for im in image_files if im.split("_")[2] == band][0]
    
    with rasterio.open(f"{img_folder}/{selected_file}") as infile:
        img = infile.read(1)
    
    return img

In [ ]:
# image_folder = "./data/"

band_dict = {
    band: get_band(image_folder, band, resolution=10)
    # band: get_band_bis(image_folder, band)
    for band in ["B02", "B03", "B04", "B08"]
}

In [ ]:
# TODO

# NDVI
# NDVI = (B8 - B4) / (B8 + B04)

# SWIR
# SWIR = (B11 - B08) / (B11 + B08)

# Nuages ? robuste
# Shapefile en sortie

In [ ]:
# B = B04, G = B03, R = B02
R = band_dict["B02"]
G = band_dict["B03"]
B = band_dict["B04"]
B8 = band_dict["B08"]
NDVI = (B8 - B) / (B8 + B)

NDVI_processed = NDVI / 255
NDVI_processed = NDVI_processed.astype(int)

plt.imshow(NDVI_processed)
plt.show()

In [ ]:
# Recreating an RGB image
img = cv2.merge((band_dict["B08"], band_dict["B04"], band_dict["B03"]))

plt.imshow(img)
plt.show()

In [ ]:
img_processed = img / 8
img_processed = img_processed.astype(int)

plt.imshow(img_processed)
plt.show()

Splitting the images to machine learning ready sizes

In [ ]:
frag_count = 45
frag_size = int(img_processed.shape[0] / frag_count)
frag_dict = {}

for y, x in itertools.product(range(frag_count), range(frag_count)):
    frag_dict[(x, y)] = img_processed[y * frag_size: (y + 1) * frag_size,
        x * frag_size: (x + 1) * frag_size, :]

Link data to the satellite images

In [ ]:
for k in range(35, 40):
    plt.imshow(frag_dict[(k, 30)])
    plt.show()

# plt.imshow(frag_dict[(20, 12)])
# plt.show()

In [ ]:
def get_tci_file_path(image_folder):
    subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    image_folder_path  = f"{image_folder}/GRANULE/{subfolder}/IMG_DATA/R10m"
    image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    selected_file = [im for im in image_files if im.split("_")[2] == "TCI"][0]
    
    return f"{image_folder_path}/{selected_file}"

tci_file_path = get_tci_file_path(image_folder)

In [ ]:
# Testing with the center of fragment 10, 10
pixel_column = int(frag_size * 10.5)
pixel_row = int(frag_size * 10.5)
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

UTM: Universal Tranverse Mercator

In [ ]:
transform = rasterio.open(tci_file_path, driver='JP2OpenJPEG').transform
zone_number = int(tci_file_path.split("/")[-1][1:3])
zone_letter = tci_file_path.split("/")[-1][0]

utm_x, utm_y = transform[2], transform[5]

In [ ]:
# Converting pixel position to UTM
east = utm_x + pixel_column * 10
north = utm_y + pixel_row * - 10

# Converting UTM to latitude and longitude
latitude, longitude = utm.to_latlon(east, north, zone_number, zone_letter)

In [ ]:
print(f"latitude, longitude: {latitude}, {longitude} ")

In [ ]:
# Converting latitude and longitude to UTM
east, north, zone_number, zone_letter = utm.from_latlon(latitude, longitude, force_zone_number=zone_number)

# Converting UTM to column and row
pixel_column = round((east - utm_x) / 10)
pixel_row = round((north - utm_y) / - 10)

In [ ]:
print(f"pixel_column: {pixel_column}")
print(f"pixel_row: {pixel_row}")

In [ ]:
plt.figure(figsize=(16, 9))
plt.subplot(221)
plt.imshow(frag_dict[(37, 30)])
plt.subplot(222)
plt.imshow(frag_dict[(38, 30)])
plt.subplot(223)
plt.imshow(frag_dict[(37, 31)])
plt.subplot(224)
plt.imshow(frag_dict[(38, 31)])
plt.savefig("test.png")
plt.show()

Edge detection with Canny

In [ ]:
img = cv2.imread("test.png")
edges = cv2.Canny(img, 100, 200)

plt.figure(figsize=(16, 9))
plt.subplot(121)
plt.imshow(img)
plt.title('Original Image')
plt.xticks([])
plt.yticks([])
plt.subplot(122)
plt.imshow(edges, cmap='gray')
plt.title('Edge Image')
plt.xticks([])
plt.yticks([])
plt.show()